In [3]:
#loading data from kaggle api
# !kaggle competitions download -c mitsui-commodity-prediction-challenge

In [5]:
#unzipping data
# !unzip mitsui-commodity-prediction-challenge.zip

In [1]:
# imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

plt.style.use('seaborn-v0_8-darkgrid')
pd.options.display.float_format = '{:,.4f}'.format

In [2]:
# 2. Load Data
target_pairs = pd.read_csv("../data/target_pairs.csv")
train_labels = pd.read_csv("../data/train_labels.csv")
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")
test_labels_lag_1 = pd.read_csv("../data/lagged_test_labels/test_labels_lag_1.csv")
test_labels_lag_2 = pd.read_csv("../data/lagged_test_labels/test_labels_lag_2.csv")
test_labels_lag_3 = pd.read_csv("../data/lagged_test_labels/test_labels_lag_3.csv")
test_labels_lag_4 = pd.read_csv("../data/lagged_test_labels/test_labels_lag_4.csv")

In [3]:
target = target_pairs.iloc[52,0]
lag = target_pairs.iloc[52,1]
pair = target_pairs.iloc[52,2]
target_pairs.iloc[52]

target                                            target_52
lag                                                       1
pair      US_Stock_CAT_adj_close - JPX_Gold_Standard_Fut...
Name: 52, dtype: object

In [4]:
commodities = pair.split(' - ')
comA = commodities[0]
comB = commodities[1]
commodities

['US_Stock_CAT_adj_close', 'JPX_Gold_Standard_Futures_Close']

In [5]:
test_df = test[["date_id",comA,comB]]
test_df

,date_id,US_Stock_CAT_adj_close,JPX_Gold_Standard_Futures_Close
0,1827,382.7670,"13,641.0000"
1,1828,NaN,"13,670.0000"
2,1829,396.4511,"13,661.0000"
3,1830,395.7047,"13,828.0000"
4,1831,404.4526,"13,934.0000"
...,...,...,...
129,1956,413.7100,"16,090.0000"
130,1957,411.5722,NaN
131,1958,418.7183,"16,221.0000"
132,1959,429.1564,"16,306.0000"


In [6]:
test_labels_lag_1[["date_id","target_52"]]

,date_id,target_52
0,1829,NaN
1,1830,-0.0140
2,1831,0.0142
3,1832,0.0046
4,1833,-0.0287
...,...,...
129,1958,NaN
130,1959,0.0194
131,1960,0.0228
132,1961,0.0101


In [7]:
test_labels_lag_1[["date_id","target_52","label_date_id"]]

,date_id,target_52,label_date_id
0,1829,NaN,1827
1,1830,-0.0140,1828
2,1831,0.0142,1829
3,1832,0.0046,1830
4,1833,-0.0287,1831
...,...,...,...
129,1958,NaN,1956
130,1959,0.0194,1957
131,1960,0.0228,1958
132,1961,0.0101,1959


In [8]:
train_df = train[["date_id",comA,comB]]
train_df

,date_id,US_Stock_CAT_adj_close,JPX_Gold_Standard_Futures_Close
0,0,132.7149,NaN
1,1,132.9177,NaN
2,2,134.7431,"4,730.0000"
3,3,136.8728,"4,780.0000"
4,4,140.3123,NaN
...,...,...,...
1956,1956,413.7100,"16,090.0000"
1957,1957,411.5722,NaN
1958,1958,418.7183,"16,221.0000"
1959,1959,429.1564,"16,306.0000"


In [9]:
first_date_test = test["date_id"].min()
last_date_test = test["date_id"].max()
first_date_test_label = test_labels_lag_1["date_id"].min()
last_date_test_label = test_labels_lag_1["date_id"].max()

start_date = first_date_test_label
last_date = last_date_test

train_df_filtered = train_df[(train_df['date_id'] >= start_date) & (train_df['date_id'] <= last_date)]
train_df_filtered

,date_id,US_Stock_CAT_adj_close,JPX_Gold_Standard_Futures_Close
1829,1829,396.4511,"13,661.0000"
1830,1830,395.7047,"13,828.0000"
1831,1831,404.4526,"13,934.0000"
1832,1832,405.6767,"13,912.0000"
1833,1833,393.0873,"13,873.0000"
...,...,...,...
1956,1956,413.7100,"16,090.0000"
1957,1957,411.5722,NaN
1958,1958,418.7183,"16,221.0000"
1959,1959,429.1564,"16,306.0000"


In [10]:
test_df = test[["date_id",comA,comB]]
test_df

,date_id,US_Stock_CAT_adj_close,JPX_Gold_Standard_Futures_Close
0,1827,382.7670,"13,641.0000"
1,1828,NaN,"13,670.0000"
2,1829,396.4511,"13,661.0000"
3,1830,395.7047,"13,828.0000"
4,1831,404.4526,"13,934.0000"
...,...,...,...
129,1956,413.7100,"16,090.0000"
130,1957,411.5722,NaN
131,1958,418.7183,"16,221.0000"
132,1959,429.1564,"16,306.0000"


In [11]:
import warnings
def generate_log_returns(data, lag):
    log_returns = pd.Series(np.nan, index=data.index)

    # Compute log returns based on the rules
    for t in range(len(data)):
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            try:
                log_returns.iloc[t] = np.log(data.iloc[t + lag + 1] / data.iloc[t + 1])
            except Exception:
                log_returns.iloc[t] = np.nan
    return log_returns


def generate_targets(date: pd.Series ,column_a: pd.Series, column_b: pd.Series, lag: int) -> pd.Series:
    a_returns = generate_log_returns(column_a, lag)
    b_returns = generate_log_returns(column_b, lag)
    return pd.DataFrame({"date_index": date, "target_generated": a_returns - b_returns})

In [12]:
train_delta = generate_targets(train["date_id"],train[comA], train[comB], lag)
train_delta

,date_index,target_generated
0,0,NaN
1,1,0.0052
2,2,NaN
3,3,NaN
4,4,0.0070
...,...,...
1956,1956,NaN
1957,1957,0.0194
1958,1958,0.0228
1959,1959,NaN


In [13]:
test_delta = generate_targets(test["date_id"],test[comA], test[comB], lag)
test_delta

,date_index,target_generated
0,1827,NaN
1,1828,-0.0140
2,1829,0.0142
3,1830,0.0046
4,1831,-0.0287
...,...,...
129,1956,NaN
130,1957,0.0194
131,1958,0.0228
132,1959,NaN


In [14]:
test_labels_lag_1[["date_id","target_0"]]

,date_id,target_0
0,1829,NaN
1,1830,0.0026
2,1831,0.0053
3,1832,0.0001
4,1833,-0.0115
...,...,...
129,1958,0.0028
130,1959,0.0114
131,1960,-0.0027
132,1961,0.0021


In [15]:
test_df_filtered = test_df[(test_df['date_id'] >= start_date) & (test_df['date_id'] <= last_date)]
test_df_filtered

,date_id,US_Stock_CAT_adj_close,JPX_Gold_Standard_Futures_Close
2,1829,396.4511,"13,661.0000"
3,1830,395.7047,"13,828.0000"
4,1831,404.4526,"13,934.0000"
5,1832,405.6767,"13,912.0000"
6,1833,393.0873,"13,873.0000"
...,...,...,...
129,1956,413.7100,"16,090.0000"
130,1957,411.5722,NaN
131,1958,418.7183,"16,221.0000"
132,1959,429.1564,"16,306.0000"


In [16]:
train_df_filtered

,date_id,US_Stock_CAT_adj_close,JPX_Gold_Standard_Futures_Close
1829,1829,396.4511,"13,661.0000"
1830,1830,395.7047,"13,828.0000"
1831,1831,404.4526,"13,934.0000"
1832,1832,405.6767,"13,912.0000"
1833,1833,393.0873,"13,873.0000"
...,...,...,...
1956,1956,413.7100,"16,090.0000"
1957,1957,411.5722,NaN
1958,1958,418.7183,"16,221.0000"
1959,1959,429.1564,"16,306.0000"


In [17]:
train_df_filtered.equals(test_df_filtered)

False

In [18]:
print(train_df_filtered.shape)
print(test_df_filtered.shape)
print(train_df_filtered.columns)
print(test_df_filtered.columns)

(132, 3)
(132, 3)
Index(['date_id', 'US_Stock_CAT_adj_close', 'JPX_Gold_Standard_Futures_Close'], dtype='object')
Index(['date_id', 'US_Stock_CAT_adj_close', 'JPX_Gold_Standard_Futures_Close'], dtype='object')


In [19]:
"""
Heteroscedasticity-Aware XGBoost Pipeline for Commodity Price Prediction
Includes: Stationarity testing, STL decomposition, volatility modeling, rolling window CV
"""

import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from statsmodels.tsa.stattools import adfuller, kpss, acf, pacf
from statsmodels.tsa.seasonal import STL
from statsmodels.stats.diagnostic import het_breuschpagan, het_white
from scipy.stats import boxcox, jarque_bera
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
import seaborn as sns

class HeteroscedasticXGBoostPipeline:
    """
    Complete pipeline for commodity price prediction with heteroscedasticity handling
    """

    def __init__(self, seasonal_period=12, max_lags=20):
        """
        Parameters:
        -----------
        seasonal_period : int
            Expected seasonality period (12 for monthly, 252 for daily trading days)
        max_lags : int
            Maximum number of lags to consider
        """
        self.seasonal_period = seasonal_period
        self.max_lags = max_lags
        self.differencing_order = 0
        self.transformation = None
        self.scaler = StandardScaler()
        self.optimal_lags = []

        # Models
        self.mean_model = None
        self.vol_model = None
        self.quantile_models = {}

        # Results storage
        self.stationarity_results = {}
        self.heteroscedasticity_results = {}
        self.stl_components = None

    def test_stationarity(self, series, alpha=0.05):
        """
        Perform ADF and KPSS tests for stationarity
        """
        print("\n=== STATIONARITY TESTS ===")

        # ADF Test (H0: unit root present - non-stationary)
        adf_result = adfuller(series.dropna(), autolag='AIC')
        print(f"\nAugmented Dickey-Fuller Test:")
        print(f"  ADF Statistic: {adf_result[0]:.4f}")
        print(f"  p-value: {adf_result[1]:.4f}")
        print(f"  Critical Values: {adf_result[4]}")
        adf_stationary = adf_result[1] < alpha
        print(f"  Result: {'STATIONARY' if adf_stationary else 'NON-STATIONARY'}")

        # KPSS Test (H0: series is stationary)
        kpss_result = kpss(series.dropna(), regression='ct', nlags='auto')
        print(f"\nKPSS Test:")
        print(f"  KPSS Statistic: {kpss_result[0]:.4f}")
        print(f"  p-value: {kpss_result[1]:.4f}")
        print(f"  Critical Values: {kpss_result[3]}")
        kpss_stationary = kpss_result[1] > alpha
        print(f"  Result: {'STATIONARY' if kpss_stationary else 'NON-STATIONARY'}")

        self.stationarity_results = {
            'adf_statistic': adf_result[0],
            'adf_pvalue': adf_result[1],
            'adf_stationary': adf_stationary,
            'kpss_statistic': kpss_result[0],
            'kpss_pvalue': kpss_result[1],
            'kpss_stationary': kpss_stationary,
            'is_stationary': adf_stationary and kpss_stationary
        }

        return self.stationarity_results['is_stationary']

    def make_stationary(self, series):
        """
        Apply transformations to make series stationary
        """
        original_series = series.copy()

        if not self.test_stationarity(series):
            print("\n=== MAKING SERIES STATIONARY ===")

            # Try log transformation first (variance stabilization)
            if (series > 0).all():
                series_log = np.log(series)
                if self.test_stationarity(series_log):
                    print("Log transformation sufficient")
                    self.transformation = 'log'
                    return series_log

            # Apply first differencing
            series_diff = series.diff().dropna()
            if self.test_stationarity(series_diff):
                print("First differencing sufficient")
                self.differencing_order = 1
                self.transformation = 'diff'
                return series_diff

            # Apply second differencing if needed
            series_diff2 = series_diff.diff().dropna()
            if self.test_stationarity(series_diff2):
                print("Second differencing required")
                self.differencing_order = 2
                self.transformation = 'diff2'
                return series_diff2

            print("Warning: Could not achieve stationarity. Proceeding with first difference.")
            self.differencing_order = 1
            self.transformation = 'diff'
            return series.diff().dropna()

        print("Series is already stationary")
        self.transformation = 'none'
        return series

    def find_optimal_lags(self, series, method='acf', threshold=0.2):
        """
        Determine optimal lag order using ACF/PACF
        """
        print("\n=== FINDING OPTIMAL LAGS ===")

        series_clean = series.dropna()

        # ACF
        acf_values = acf(series_clean, nlags=self.max_lags, fft=False)
        significant_acf = np.where(np.abs(acf_values[1:]) > threshold)[0] + 1

        # PACF
        pacf_values = pacf(series_clean, nlags=self.max_lags, method='ols')
        significant_pacf = np.where(np.abs(pacf_values[1:]) > threshold)[0] + 1

        if method == 'acf':
            self.optimal_lags = significant_acf[:10].tolist()  # Top 10 lags
        elif method == 'pacf':
            self.optimal_lags = significant_pacf[:10].tolist()
        else:  # both
            self.optimal_lags = sorted(list(set(significant_acf[:10].tolist() +
                                                 significant_pacf[:10].tolist())))

        if not self.optimal_lags:
            self.optimal_lags = [1, 2, 3, 5, 7]  # Default lags

        print(f"Optimal lags: {self.optimal_lags}")
        return self.optimal_lags

    def stl_decomposition(self, series):
        """
        Perform STL decomposition
        """
        print("\n=== STL DECOMPOSITION ===")

        series_clean = series.dropna()

        if len(series_clean) < 2 * self.seasonal_period:
            print(f"Warning: Series too short for seasonal decomposition (need >= {2*self.seasonal_period})")
            return None

        stl = STL(series_clean, seasonal=self.seasonal_period, robust=True)
        result = stl.fit()

        self.stl_components = {
            'trend': result.trend,
            'seasonal': result.seasonal,
            'resid': result.resid
        }

        print(f"Trend strength: {1 - result.resid.var() / (result.trend + result.resid).var():.4f}")
        print(f"Seasonal strength: {1 - result.resid.var() / (result.seasonal + result.resid).var():.4f}")

        return self.stl_components

    def test_heteroscedasticity(self, residuals, features):
        """
        Test for heteroscedasticity using Breusch-Pagan and White tests
        """
        print("\n=== HETEROSCEDASTICITY TESTS ===")

        # Breusch-Pagan test
        try:
            bp_stat, bp_pval, _, _ = het_breuschpagan(residuals, features)
            print(f"\nBreusch-Pagan Test:")
            print(f"  LM Statistic: {bp_stat:.4f}")
            print(f"  p-value: {bp_pval:.4f}")
            bp_hetero = bp_pval < 0.05
            print(f"  Result: {'HETEROSCEDASTIC' if bp_hetero else 'HOMOSCEDASTIC'}")
        except:
            bp_hetero = True
            bp_pval = 0.0
            print("Breusch-Pagan test failed")

        # White test
        try:
            white_stat, white_pval, _, _ = het_white(residuals, features)
            print(f"\nWhite Test:")
            print(f"  LM Statistic: {white_stat:.4f}")
            print(f"  p-value: {white_pval:.4f}")
            white_hetero = white_pval < 0.05
            print(f"  Result: {'HETEROSCEDASTIC' if white_hetero else 'HOMOSCEDASTIC'}")
        except:
            white_hetero = True
            white_pval = 0.0
            print("White test failed")

        self.heteroscedasticity_results = {
            'bp_pvalue': bp_pval,
            'white_pvalue': white_pval,
            'is_heteroscedastic': bp_hetero or white_hetero
        }

        return self.heteroscedasticity_results['is_heteroscedastic']

    def create_features(self, series, include_volatility=True):
        """
        Create feature matrix with lags, rolling statistics, and volatility features
        """
        df = pd.DataFrame({'target': series})

        # Lag features
        for lag in self.optimal_lags:
            df[f'lag_{lag}'] = df['target'].shift(lag)

        # Rolling statistics
        for window in [5, 10, 20]:
            df[f'rolling_mean_{window}'] = df['target'].rolling(window).mean()
            df[f'rolling_std_{window}'] = df['target'].rolling(window).std()
            df[f'rolling_min_{window}'] = df['target'].rolling(window).min()
            df[f'rolling_max_{window}'] = df['target'].rolling(window).max()

        # Volatility features (if enabled)
        if include_volatility:
            # Historical volatility at multiple horizons
            for window in [5, 10, 20, 60]:
                df[f'volatility_{window}'] = df['target'].rolling(window).std()

            # Realized volatility (if returns)
            df['realized_vol'] = df['target'].rolling(20).std() * np.sqrt(252)

            # Volatility of volatility
            df['vol_of_vol'] = df['volatility_20'].rolling(20).std()

            # Volatility regime (high/low)
            vol_median = df['volatility_20'].rolling(60).median()
            df['high_vol_regime'] = (df['volatility_20'] > vol_median).astype(int)

            # Range-based volatility proxy
            df['range_vol'] = (df['rolling_max_20'] - df['rolling_min_20']) / df['rolling_mean_20']

        # STL components (if available)
        if self.stl_components is not None:
            df['trend'] = self.stl_components['trend']
            df['seasonal'] = self.stl_components['seasonal']

        # Time features
        if isinstance(df.index, pd.DatetimeIndex):
            df['day_of_week'] = df.index.dayofweek
            df['month'] = df.index.month
            df['quarter'] = df.index.quarter
            df['day_of_month'] = df.index.day

        # Drop NaN rows
        df = df.dropna()

        return df

    def create_volatility_weights(self, series, window=20, method='inverse_variance'):
        """
        Create sample weights based on volatility
        """
        if method == 'inverse_variance':
            rolling_var = series.rolling(window).var()
            weights = 1 / (rolling_var + 1e-8)  # Add small constant to avoid division by zero
        elif method == 'exponential':
            half_life = window
            weights = np.exp(-np.log(2) * np.arange(len(series))[::-1] / half_life)
        else:
            weights = np.ones(len(series))

        weights = weights / weights.sum() * len(series)  # Normalize to sum to n
        return weights

    def rolling_window_cv(self, data, target_col='target',
                          initial_train_size=None, step_size=1,
                          use_quantile=False, quantiles=[0.1, 0.5, 0.9]):
        """
        Perform rolling window cross-validation

        Parameters:
        -----------
        data : DataFrame
            Feature matrix with target column
        target_col : str
            Name of target column
        initial_train_size : int or None
            Initial training window size (if None, uses one seasonal period)
        step_size : int
            Number of periods to roll forward
        use_quantile : bool
            Whether to use quantile regression
        quantiles : list
            Quantiles to predict if use_quantile=True
        """
        print("\n=== ROLLING WINDOW CROSS-VALIDATION ===")

        if initial_train_size is None:
            initial_train_size = self.seasonal_period

        feature_cols = [col for col in data.columns if col != target_col]
        X = data[feature_cols].values
        y = data[target_col].values

        predictions = []
        actuals = []
        prediction_intervals = []
        fold_metrics = []

        n_folds = (len(data) - initial_train_size) // step_size

        for fold in range(n_folds):
            train_end = initial_train_size + fold * step_size
            test_start = train_end
            test_end = min(test_start + step_size, len(data))

            if test_end >= len(data):
                break

            X_train, y_train = X[:train_end], y[:train_end]
            X_test, y_test = X[test_start:test_end], y[test_start:test_end]

            print(f"\nFold {fold + 1}/{n_folds}: Train size={len(X_train)}, Test size={len(X_test)}")

            # Create volatility-based sample weights
            weights = self.create_volatility_weights(
                pd.Series(y_train),
                method='inverse_variance'
            )

            if use_quantile:
                # Quantile regression approach
                fold_predictions = {}

                for q in quantiles:
                    model = XGBRegressor(
                        objective='reg:quantileerror',
                        quantile_alpha=q,
                        n_estimators=100,
                        max_depth=4,
                        learning_rate=0.05,
                        subsample=0.8,
                        colsample_bytree=0.8,
                        random_state=42
                    )
                    model.fit(X_train, y_train, sample_weight=weights, verbose=False)
                    fold_predictions[q] = model.predict(X_test)

                # Use median as point prediction
                pred = fold_predictions[0.5]
                lower = fold_predictions[quantiles[0]]
                upper = fold_predictions[quantiles[-1]]

                prediction_intervals.extend(list(zip(lower, upper)))

            else:
                # Two-model approach: Mean + Volatility

                # Model 1: Mean prediction
                mean_model = XGBRegressor(
                    n_estimators=100,
                    max_depth=4,
                    learning_rate=0.05,
                    subsample=0.8,
                    colsample_bytree=0.8,
                    random_state=42
                )
                mean_model.fit(X_train, y_train, sample_weight=weights, verbose=False)
                pred_mean = mean_model.predict(X_test)

                # Model 2: Volatility prediction
                train_residuals = y_train - mean_model.predict(X_train)
                train_vol = pd.Series(train_residuals).rolling(20, min_periods=5).std().fillna(
                    pd.Series(train_residuals).std()
                ).values

                vol_model = XGBRegressor(
                    n_estimators=50,
                    max_depth=3,
                    learning_rate=0.05,
                    subsample=0.8,
                    colsample_bytree=0.8,
                    random_state=42
                )
                vol_model.fit(X_train, train_vol, verbose=False)
                pred_vol = vol_model.predict(X_test)

                pred = pred_mean
                lower = pred_mean - 1.96 * pred_vol
                upper = pred_mean + 1.96 * pred_vol

                prediction_intervals.extend(list(zip(lower, upper)))

            predictions.extend(pred)
            actuals.extend(y_test)

            # Calculate fold metrics
            rmse = np.sqrt(np.mean((pred - y_test) ** 2))
            mae = np.mean(np.abs(pred - y_test))

            # Coverage of prediction intervals
            in_interval = np.sum((y_test >= lower) & (y_test <= upper))
            coverage = in_interval / len(y_test)

            fold_metrics.append({
                'fold': fold + 1,
                'rmse': rmse,
                'mae': mae,
                'coverage': coverage
            })

            print(f"  RMSE: {rmse:.4f}, MAE: {mae:.4f}, Coverage: {coverage:.2%}")

        results = {
            'predictions': np.array(predictions),
            'actuals': np.array(actuals),
            'prediction_intervals': prediction_intervals,
            'fold_metrics': pd.DataFrame(fold_metrics)
        }

        # Overall metrics
        overall_rmse = np.sqrt(np.mean((results['predictions'] - results['actuals']) ** 2))
        overall_mae = np.mean(np.abs(results['predictions'] - results['actuals']))

        print(f"\n=== OVERALL CROSS-VALIDATION RESULTS ===")
        print(f"RMSE: {overall_rmse:.4f}")
        print(f"MAE: {overall_mae:.4f}")
        print(f"Mean Coverage: {results['fold_metrics']['coverage'].mean():.2%}")

        return results

    def fit(self, series, use_quantile=False):
        """
        Complete fitting pipeline
        """
        print("="*60)
        print("STARTING HETEROSCEDASTICITY-AWARE XGBOOST PIPELINE")
        print("="*60)

        # Step 1: Make stationary
        series_stationary = self.make_stationary(series)

        # Step 2: Find optimal lags
        self.find_optimal_lags(series_stationary)

        # Step 3: STL decomposition
        self.stl_decomposition(series_stationary)

        # Step 4: Create features
        features_df = self.create_features(series_stationary, include_volatility=True)

        # Step 5: Test heteroscedasticity (on a preliminary model)
        X_temp = features_df.drop('target', axis=1).values
        y_temp = features_df['target'].values

        temp_model = XGBRegressor(n_estimators=50, max_depth=3, random_state=42)
        temp_model.fit(X_temp[:len(X_temp)//2], y_temp[:len(y_temp)//2], verbose=False)
        temp_residuals = y_temp[:len(y_temp)//2] - temp_model.predict(X_temp[:len(X_temp)//2])

        is_hetero = self.test_heteroscedasticity(temp_residuals, X_temp[:len(X_temp)//2])

        if is_hetero:
            print("\n*** HETEROSCEDASTICITY DETECTED - Using volatility modeling ***")

        # Step 6: Rolling window cross-validation
        cv_results = self.rolling_window_cv(
            features_df,
            use_quantile=use_quantile,
            initial_train_size=self.seasonal_period
        )

        return cv_results


# Example usage
if __name__ == "__main__":
    # Generate synthetic commodity price data with heteroscedasticity
    np.random.seed(42)
    n = 500
    t = np.arange(n)

    # Trend + Seasonality + Heteroscedastic noise
    trend = 0.05 * t
    seasonal = 10 * np.sin(2 * np.pi * t / 12)

    # Heteroscedastic volatility (GARCH-like)
    volatility = np.zeros(n)
    volatility[0] = 1.0
    for i in range(1, n):
        volatility[i] = 0.1 + 0.85 * volatility[i-1] + 0.1 * np.random.randn()**2

    noise = volatility * np.random.randn(n)
    price = 100 + trend + seasonal + noise

    # Create time series
    dates = pd.date_range('2020-01-01', periods=n, freq='D')
    series = pd.Series(price, index=dates, name='price')

    print("Sample data shape:", series.shape)
    print("Sample data (first 5):", series.head())

    # Initialize and fit pipeline
    pipeline = HeteroscedasticXGBoostPipeline(seasonal_period=12, max_lags=20)

    # Option 1: Quantile regression (recommended for heteroscedasticity)
    results_quantile = pipeline.fit(series, use_quantile=True)

    # Option 2: Two-model approach (mean + volatility)
    # results_two_model = pipeline.fit(series, use_quantile=False)

    print("\n" + "="*60)
    print("PIPELINE COMPLETE")
    print("="*60)

Sample data shape: (500,)
Sample data (first 5): 2020-01-01    98.6172
2020-01-02   105.9527
2020-01-03   110.5367
2020-01-04   108.8454
2020-01-05   109.4935
Freq: D, Name: price, dtype: float64
STARTING HETEROSCEDASTICITY-AWARE XGBOOST PIPELINE

=== STATIONARITY TESTS ===

Augmented Dickey-Fuller Test:
  ADF Statistic: -0.5331
  p-value: 0.8854
  Critical Values: {'1%': np.float64(-3.4438771098680196), '5%': np.float64(-2.867505393939065), '10%': np.float64(-2.569947324764179)}
  Result: NON-STATIONARY

KPSS Test:
  KPSS Statistic: 0.0220
  p-value: 0.1000
  Critical Values: {'10%': 0.119, '5%': 0.146, '2.5%': 0.176, '1%': 0.216}
  Result: STATIONARY

=== MAKING SERIES STATIONARY ===

=== STATIONARITY TESTS ===

Augmented Dickey-Fuller Test:
  ADF Statistic: -0.9713
  p-value: 0.7636
  Critical Values: {'1%': np.float64(-3.4438771098680196), '5%': np.float64(-2.867505393939065), '10%': np.float64(-2.569947324764179)}
  Result: NON-STATIONARY

KPSS Test:
  KPSS Statistic: 0.0413
  p-v

ValueError: seasonal must be an odd positive integer >= 3

In [ ]:
"""
Heteroscedasticity-Aware XGBoost Pipeline for Commodity Price Prediction
Includes: Stationarity testing, STL decomposition, volatility modeling, rolling window CV
"""

import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from statsmodels.tsa.stattools import adfuller, kpss, acf, pacf
from statsmodels.tsa.seasonal import STL
from statsmodels.stats.diagnostic import het_breuschpagan, het_white
from scipy.stats import boxcox, jarque_bera
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
import seaborn as sns

class HeteroscedasticXGBoostPipeline:
    """
    Complete pipeline for commodity price prediction with heteroscedasticity handling
    """

    def __init__(self, seasonal_period=12, max_lags=20):
        """
        Parameters:
        -----------
        seasonal_period : int
            Expected seasonality period (12 for monthly, 252 for daily trading days)
            Note: Will be adjusted to odd number >= 3 for STL decomposition
        max_lags : int
            Maximum number of lags to consider
        """
        self.seasonal_period = seasonal_period
        self.max_lags = max_lags
        self.differencing_order = 0
        self.transformation = None
        self.scaler = StandardScaler()
        self.optimal_lags = []

        # Models
        self.mean_model = None
        self.vol_model = None
        self.quantile_models = {}

        # Results storage
        self.stationarity_results = {}
        self.heteroscedasticity_results = {}
        self.stl_components = None

    def test_stationarity(self, series, alpha=0.05):
        """
        Perform ADF and KPSS tests for stationarity
        """
        print("\n=== STATIONARITY TESTS ===")

        # ADF Test (H0: unit root present - non-stationary)
        adf_result = adfuller(series.dropna(), autolag='AIC')
        print(f"\nAugmented Dickey-Fuller Test:")
        print(f"  ADF Statistic: {adf_result[0]:.4f}")
        print(f"  p-value: {adf_result[1]:.4f}")
        print(f"  Critical Values: {adf_result[4]}")
        adf_stationary = adf_result[1] < alpha
        print(f"  Result: {'STATIONARY' if adf_stationary else 'NON-STATIONARY'}")

        # KPSS Test (H0: series is stationary)
        kpss_result = kpss(series.dropna(), regression='ct', nlags='auto')
        print(f"\nKPSS Test:")
        print(f"  KPSS Statistic: {kpss_result[0]:.4f}")
        print(f"  p-value: {kpss_result[1]:.4f}")
        print(f"  Critical Values: {kpss_result[3]}")
        kpss_stationary = kpss_result[1] > alpha
        print(f"  Result: {'STATIONARY' if kpss_stationary else 'NON-STATIONARY'}")

        self.stationarity_results = {
            'adf_statistic': adf_result[0],
            'adf_pvalue': adf_result[1],
            'adf_stationary': adf_stationary,
            'kpss_statistic': kpss_result[0],
            'kpss_pvalue': kpss_result[1],
            'kpss_stationary': kpss_stationary,
            'is_stationary': adf_stationary and kpss_stationary
        }

        return self.stationarity_results['is_stationary']

    def make_stationary(self, series):
        """
        Apply transformations to make series stationary
        """
        original_series = series.copy()

        if not self.test_stationarity(series):
            print("\n=== MAKING SERIES STATIONARY ===")

            # Try log transformation first (variance stabilization)
            if (series > 0).all():
                series_log = np.log(series)
                if self.test_stationarity(series_log):
                    print("Log transformation sufficient")
                    self.transformation = 'log'
                    return series_log

            # Apply first differencing
            series_diff = series.diff().dropna()
            if self.test_stationarity(series_diff):
                print("First differencing sufficient")
                self.differencing_order = 1
                self.transformation = 'diff'
                return series_diff

            # Apply second differencing if needed
            series_diff2 = series_diff.diff().dropna()
            if self.test_stationarity(series_diff2):
                print("Second differencing required")
                self.differencing_order = 2
                self.transformation = 'diff2'
                return series_diff2

            print("Warning: Could not achieve stationarity. Proceeding with first difference.")
            self.differencing_order = 1
            self.transformation = 'diff'
            return series.diff().dropna()

        print("Series is already stationary")
        self.transformation = 'none'
        return series

    def find_optimal_lags(self, series, method='acf', threshold=0.2):
        """
        Determine optimal lag order using ACF/PACF
        """
        print("\n=== FINDING OPTIMAL LAGS ===")

        series_clean = series.dropna()

        # ACF
        acf_values = acf(series_clean, nlags=self.max_lags, fft=False)
        significant_acf = np.where(np.abs(acf_values[1:]) > threshold)[0] + 1

        # PACF
        pacf_values = pacf(series_clean, nlags=self.max_lags, method='ols')
        significant_pacf = np.where(np.abs(pacf_values[1:]) > threshold)[0] + 1

        if method == 'acf':
            self.optimal_lags = significant_acf[:10].tolist()  # Top 10 lags
        elif method == 'pacf':
            self.optimal_lags = significant_pacf[:10].tolist()
        else:  # both
            self.optimal_lags = sorted(list(set(significant_acf[:10].tolist() +
                                                 significant_pacf[:10].tolist())))

        if not self.optimal_lags:
            self.optimal_lags = [1, 2, 3, 5, 7]  # Default lags

        print(f"Optimal lags: {self.optimal_lags}")
        return self.optimal_lags

    def stl_decomposition(self, series):
        """
        Perform STL decomposition
        """
        print("\n=== STL DECOMPOSITION ===")

        series_clean = series.dropna()

        if len(series_clean) < 2 * self.seasonal_period:
            print(f"Warning: Series too short for seasonal decomposition (need >= {2*self.seasonal_period})")
            return None

        # Ensure seasonal parameter is odd and >= 3
        seasonal_param = self.seasonal_period
        if seasonal_param % 2 == 0:
            seasonal_param += 1
        if seasonal_param < 3:
            seasonal_param = 3

        print(f"Using seasonal parameter: {seasonal_param}")

        try:
            stl = STL(series_clean, seasonal=seasonal_param, robust=True)
            result = stl.fit()
        except Exception as e:
            print(f"STL decomposition failed: {e}")
            print("Proceeding without STL components")
            return None

        self.stl_components = {
            'trend': result.trend,
            'seasonal': result.seasonal,
            'resid': result.resid
        }

        print(f"Trend strength: {1 - result.resid.var() / (result.trend + result.resid).var():.4f}")
        print(f"Seasonal strength: {1 - result.resid.var() / (result.seasonal + result.resid).var():.4f}")

        return self.stl_components

    def test_heteroscedasticity(self, residuals, features):
        """
        Test for heteroscedasticity using Breusch-Pagan and White tests
        """
        print("\n=== HETEROSCEDASTICITY TESTS ===")

        # Breusch-Pagan test
        try:
            bp_stat, bp_pval, _, _ = het_breuschpagan(residuals, features)
            print(f"\nBreusch-Pagan Test:")
            print(f"  LM Statistic: {bp_stat:.4f}")
            print(f"  p-value: {bp_pval:.4f}")
            bp_hetero = bp_pval < 0.05
            print(f"  Result: {'HETEROSCEDASTIC' if bp_hetero else 'HOMOSCEDASTIC'}")
        except:
            bp_hetero = True
            bp_pval = 0.0
            print("Breusch-Pagan test failed")

        # White test
        try:
            white_stat, white_pval, _, _ = het_white(residuals, features)
            print(f"\nWhite Test:")
            print(f"  LM Statistic: {white_stat:.4f}")
            print(f"  p-value: {white_pval:.4f}")
            white_hetero = white_pval < 0.05
            print(f"  Result: {'HETEROSCEDASTIC' if white_hetero else 'HOMOSCEDASTIC'}")
        except:
            white_hetero = True
            white_pval = 0.0
            print("White test failed")

        self.heteroscedasticity_results = {
            'bp_pvalue': bp_pval,
            'white_pvalue': white_pval,
            'is_heteroscedastic': bp_hetero or white_hetero
        }

        return self.heteroscedasticity_results['is_heteroscedastic']

    def create_features(self, series, include_volatility=True):
        """
        Create feature matrix with lags, rolling statistics, and volatility features
        """
        df = pd.DataFrame({'target': series})

        # Lag features
        for lag in self.optimal_lags:
            df[f'lag_{lag}'] = df['target'].shift(lag)

        # Rolling statistics
        for window in [5, 10, 20]:
            df[f'rolling_mean_{window}'] = df['target'].rolling(window).mean()
            df[f'rolling_std_{window}'] = df['target'].rolling(window).std()
            df[f'rolling_min_{window}'] = df['target'].rolling(window).min()
            df[f'rolling_max_{window}'] = df['target'].rolling(window).max()

        # Volatility features (if enabled)
        if include_volatility:
            # Historical volatility at multiple horizons
            for window in [5, 10, 20, 60]:
                df[f'volatility_{window}'] = df['target'].rolling(window).std()

            # Realized volatility (if returns)
            df['realized_vol'] = df['target'].rolling(20).std() * np.sqrt(252)

            # Volatility of volatility
            df['vol_of_vol'] = df['volatility_20'].rolling(20).std()

            # Volatility regime (high/low)
            vol_median = df['volatility_20'].rolling(60).median()
            df['high_vol_regime'] = (df['volatility_20'] > vol_median).astype(int)

            # Range-based volatility proxy
            df['range_vol'] = (df['rolling_max_20'] - df['rolling_min_20']) / df['rolling_mean_20']

        # STL components (if available)
        if self.stl_components is not None:
            df['trend'] = self.stl_components['trend']
            df['seasonal'] = self.stl_components['seasonal']

        # Time features
        if isinstance(df.index, pd.DatetimeIndex):
            df['day_of_week'] = df.index.dayofweek
            df['month'] = df.index.month
            df['quarter'] = df.index.quarter
            df['day_of_month'] = df.index.day

        # Drop NaN rows
        df = df.dropna()

        return df

    def create_volatility_weights(self, series, window=20, method='inverse_variance'):
        """
        Create sample weights based on volatility
        """
        # Convert to numpy array if pandas Series
        if isinstance(series, pd.Series):
            values = series.values
        else:
            values = np.array(series)

        if method == 'inverse_variance':
            # Calculate rolling variance manually to avoid pandas issues
            weights = np.ones(len(values))
            for i in range(window, len(values)):
                window_data = values[i-window:i]
                var = np.var(window_data)
                if var > 1e-10:  # Only if variance is meaningful
                    weights[i] = 1.0 / var
                else:
                    weights[i] = 1.0

            # Fill first window values with mean weight
            mean_weight = np.mean(weights[window:])
            weights[:window] = mean_weight

        elif method == 'exponential':
            half_life = window
            weights = np.exp(-np.log(2) * np.arange(len(values))[::-1] / half_life)
        else:
            weights = np.ones(len(values))

        # Ensure all weights are strictly positive
        weights = np.abs(weights)  # Remove any negative
        weights = np.where(weights < 1e-8, 1e-8, weights)  # Floor at small positive
        weights = np.where(np.isnan(weights), 1.0, weights)  # Replace NaN
        weights = np.where(np.isinf(weights), 1.0, weights)  # Replace Inf

        # Normalize to sum to n (standard for XGBoost)
        weights = weights / np.sum(weights) * len(weights)

        # Final safety check
        assert np.all(weights > 0), "All weights must be positive"
        assert np.all(np.isfinite(weights)), "All weights must be finite"

        return weights

    def rolling_window_cv(self, data, target_col='target',
                          initial_train_size=None, step_size=1,
                          use_quantile=False, quantiles=[0.1, 0.5, 0.9]):
        """
        Perform rolling window cross-validation

        Parameters:
        -----------
        data : DataFrame
            Feature matrix with target column
        target_col : str
            Name of target column
        initial_train_size : int or None
            Initial training window size (if None, uses one seasonal period)
        step_size : int
            Number of periods to roll forward
        use_quantile : bool
            Whether to use quantile regression
        quantiles : list
            Quantiles to predict if use_quantile=True
        """
        print("\n=== ROLLING WINDOW CROSS-VALIDATION ===")

        if initial_train_size is None:
            initial_train_size = self.seasonal_period

        feature_cols = [col for col in data.columns if col != target_col]
        X = data[feature_cols].values
        y = data[target_col].values

        predictions = []
        actuals = []
        prediction_intervals = []
        fold_metrics = []

        n_folds = (len(data) - initial_train_size) // step_size

        for fold in range(n_folds):
            train_end = initial_train_size + fold * step_size
            test_start = train_end
            test_end = min(test_start + step_size, len(data))

            if test_end >= len(data):
                break

            X_train, y_train = X[:train_end], y[:train_end]
            X_test, y_test = X[test_start:test_end], y[test_start:test_end]

            print(f"\nFold {fold + 1}/{n_folds}: Train size={len(X_train)}, Test size={len(X_test)}")

            # Create volatility-based sample weights
            try:
                weights = self.create_volatility_weights(
                    y_train,  # Pass numpy array directly
                    window=min(20, len(y_train)//2),  # Adjust window for small samples
                    method='inverse_variance'
                )

                # Validate weights
                assert len(weights) == len(y_train), f"Weight length mismatch: {len(weights)} vs {len(y_train)}"
                assert np.all(weights > 0), "Some weights are non-positive"
                assert np.all(np.isfinite(weights)), "Some weights are not finite"

            except Exception as e:
                print(f"  Warning: Weight creation failed ({e}). Using uniform weights.")
                weights = np.ones(len(y_train))

            if use_quantile:
                # Quantile regression approach
                fold_predictions = {}

                for q in quantiles:
                    model = XGBRegressor(
                        objective='reg:quantileerror',
                        quantile_alpha=q,
                        n_estimators=100,
                        max_depth=4,
                        learning_rate=0.05,
                        subsample=0.8,
                        colsample_bytree=0.8,
                        random_state=42
                    )
                    model.fit(X_train, y_train, sample_weight=weights, verbose=False)
                    fold_predictions[q] = model.predict(X_test)

                # Use median as point prediction
                pred = fold_predictions[0.5]
                lower = fold_predictions[quantiles[0]]
                upper = fold_predictions[quantiles[-1]]

                prediction_intervals.extend(list(zip(lower, upper)))

            else:
                # Two-model approach: Mean + Volatility

                # Model 1: Mean prediction
                mean_model = XGBRegressor(
                    n_estimators=100,
                    max_depth=4,
                    learning_rate=0.05,
                    subsample=0.8,
                    colsample_bytree=0.8,
                    random_state=42
                )
                mean_model.fit(X_train, y_train, sample_weight=weights, verbose=False)
                pred_mean = mean_model.predict(X_test)

                # Model 2: Volatility prediction
                train_residuals = y_train - mean_model.predict(X_train)
                train_vol = pd.Series(train_residuals).rolling(20, min_periods=5).std().fillna(
                    pd.Series(train_residuals).std()
                ).values

                vol_model = XGBRegressor(
                    n_estimators=50,
                    max_depth=3,
                    learning_rate=0.05,
                    subsample=0.8,
                    colsample_bytree=0.8,
                    random_state=42
                )
                vol_model.fit(X_train, train_vol, verbose=False)
                pred_vol = vol_model.predict(X_test)

                pred = pred_mean
                lower = pred_mean - 1.96 * pred_vol
                upper = pred_mean + 1.96 * pred_vol

                prediction_intervals.extend(list(zip(lower, upper)))

            predictions.extend(pred)
            actuals.extend(y_test)

            # Calculate fold metrics
            rmse = np.sqrt(np.mean((pred - y_test) ** 2))
            mae = np.mean(np.abs(pred - y_test))

            # Coverage of prediction intervals
            in_interval = np.sum((y_test >= lower) & (y_test <= upper))
            coverage = in_interval / len(y_test)

            fold_metrics.append({
                'fold': fold + 1,
                'rmse': rmse,
                'mae': mae,
                'coverage': coverage
            })

            print(f"  RMSE: {rmse:.4f}, MAE: {mae:.4f}, Coverage: {coverage:.2%}")

        results = {
            'predictions': np.array(predictions),
            'actuals': np.array(actuals),
            'prediction_intervals': prediction_intervals,
            'fold_metrics': pd.DataFrame(fold_metrics)
        }

        # Overall metrics
        overall_rmse = np.sqrt(np.mean((results['predictions'] - results['actuals']) ** 2))
        overall_mae = np.mean(np.abs(results['predictions'] - results['actuals']))

        print(f"\n=== OVERALL CROSS-VALIDATION RESULTS ===")
        print(f"RMSE: {overall_rmse:.4f}")
        print(f"MAE: {overall_mae:.4f}")
        print(f"Mean Coverage: {results['fold_metrics']['coverage'].mean():.2%}")

        return results

    def plot_diagnostics(self, original_series, cv_results, save_path=None):
        """
        Comprehensive diagnostic plots for model evaluation

        Parameters:
        -----------
        original_series : pd.Series
            Original time series data
        cv_results : dict
            Results from rolling_window_cv
        save_path : str, optional
            Path to save figures
        """
        fig = plt.figure(figsize=(20, 16))
        gs = fig.add_gridspec(5, 3, hspace=0.3, wspace=0.3)

        # 1. Original Series with Trend and Seasonality
        ax1 = fig.add_subplot(gs[0, :])
        ax1.plot(original_series.index, original_series.values,
                 label='Original', alpha=0.7, linewidth=1)
        if self.stl_components is not None:
            ax1.plot(self.stl_components['trend'].index,
                    self.stl_components['trend'].values,
                    label='Trend', linewidth=2, color='red')
        ax1.set_title('Original Time Series with Trend', fontsize=14, fontweight='bold')
        ax1.set_xlabel('Time')
        ax1.set_ylabel('Price')
        ax1.legend()
        ax1.grid(True, alpha=0.3)

        # 2. STL Decomposition Components
        if self.stl_components is not None:
            ax2 = fig.add_subplot(gs[1, 0])
            ax2.plot(self.stl_components['trend'].index,
                    self.stl_components['trend'].values, color='red')
            ax2.set_title('Trend Component', fontsize=12, fontweight='bold')
            ax2.grid(True, alpha=0.3)

            ax3 = fig.add_subplot(gs[1, 1])
            ax3.plot(self.stl_components['seasonal'].index,
                    self.stl_components['seasonal'].values, color='green')
            ax3.set_title('Seasonal Component', fontsize=12, fontweight='bold')
            ax3.grid(True, alpha=0.3)

            ax4 = fig.add_subplot(gs[1, 2])
            ax4.plot(self.stl_components['resid'].index,
                    self.stl_components['resid'].values, color='purple', alpha=0.6)
            ax4.set_title('Residual Component', fontsize=12, fontweight='bold')
            ax4.grid(True, alpha=0.3)

        # 3. Predictions vs Actuals
        ax5 = fig.add_subplot(gs[2, :])
        test_indices = range(len(cv_results['actuals']))
        ax5.plot(test_indices, cv_results['actuals'],
                label='Actual', color='blue', linewidth=2, alpha=0.7)
        ax5.plot(test_indices, cv_results['predictions'],
                label='Predicted', color='red', linewidth=2, alpha=0.7)

        # Add prediction intervals
        if cv_results['prediction_intervals']:
            intervals = np.array(cv_results['prediction_intervals'])
            ax5.fill_between(test_indices, intervals[:, 0], intervals[:, 1],
                           alpha=0.2, color='red', label='95% Prediction Interval')

        ax5.set_title('Predictions vs Actuals (Cross-Validation)',
                     fontsize=14, fontweight='bold')
        ax5.set_xlabel('Test Sample Index')
        ax5.set_ylabel('Value')
        ax5.legend()
        ax5.grid(True, alpha=0.3)

        # 4. Residuals Analysis
        residuals = cv_results['predictions'] - cv_results['actuals']

        ax6 = fig.add_subplot(gs[3, 0])
        ax6.scatter(cv_results['predictions'], residuals, alpha=0.5, s=20)
        ax6.axhline(y=0, color='red', linestyle='--', linewidth=2)
        ax6.set_title('Residual Plot', fontsize=12, fontweight='bold')
        ax6.set_xlabel('Predicted Values')
        ax6.set_ylabel('Residuals')
        ax6.grid(True, alpha=0.3)

        # Add LOESS smooth to detect patterns
        from scipy.signal import savgol_filter
        if len(residuals) > 50:
            sorted_idx = np.argsort(cv_results['predictions'])
            smoothed = savgol_filter(residuals[sorted_idx],
                                    min(51, len(residuals)//2*2-1), 3)
            ax6.plot(cv_results['predictions'][sorted_idx], smoothed,
                    color='orange', linewidth=2, label='Smoothed')
            ax6.legend()

        # 5. Residuals Histogram
        ax7 = fig.add_subplot(gs[3, 1])
        ax7.hist(residuals, bins=30, edgecolor='black', alpha=0.7)
        ax7.axvline(x=0, color='red', linestyle='--', linewidth=2)
        ax7.set_title('Residuals Distribution', fontsize=12, fontweight='bold')
        ax7.set_xlabel('Residual Value')
        ax7.set_ylabel('Frequency')

        # Add normality statistics
        from scipy.stats import shapiro
        _, p_value = shapiro(residuals)
        ax7.text(0.05, 0.95, f'Shapiro-Wilk p={p_value:.4f}',
                transform=ax7.transAxes, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
        ax7.grid(True, alpha=0.3)

        # 6. Q-Q Plot
        ax8 = fig.add_subplot(gs[3, 2])
        from scipy.stats import probplot
        probplot(residuals, dist="norm", plot=ax8)
        ax8.set_title('Q-Q Plot', fontsize=12, fontweight='bold')
        ax8.grid(True, alpha=0.3)

        # 7. ACF of Residuals
        ax9 = fig.add_subplot(gs[4, 0])
        from statsmodels.graphics.tsaplots import plot_acf
        plot_acf(residuals, lags=min(40, len(residuals)//2), ax=ax9, alpha=0.05)
        ax9.set_title('ACF of Residuals', fontsize=12, fontweight='bold')
        ax9.set_xlabel('Lag')

        # 8. Rolling RMSE
        ax10 = fig.add_subplot(gs[4, 1])
        fold_metrics = cv_results['fold_metrics']
        ax10.plot(fold_metrics['fold'], fold_metrics['rmse'],
                 marker='o', linewidth=2, markersize=8)
        ax10.set_title('RMSE by Fold', fontsize=12, fontweight='bold')
        ax10.set_xlabel('Fold')
        ax10.set_ylabel('RMSE')
        ax10.grid(True, alpha=0.3)

        # 9. Coverage Analysis
        ax11 = fig.add_subplot(gs[4, 2])
        ax11.plot(fold_metrics['fold'], fold_metrics['coverage'],
                 marker='s', linewidth=2, markersize=8, color='green')
        ax11.axhline(y=0.95, color='red', linestyle='--', linewidth=2,
                    label='Target 95%')
        ax11.set_title('Prediction Interval Coverage', fontsize=12, fontweight='bold')
        ax11.set_xlabel('Fold')
        ax11.set_ylabel('Coverage')
        ax11.set_ylim([0, 1])
        ax11.legend()
        ax11.grid(True, alpha=0.3)

        plt.suptitle('Comprehensive Model Diagnostics',
                    fontsize=16, fontweight='bold', y=0.995)

        if save_path:
            plt.savefig(save_path, dpi=300, bbox_inches='tight')
            print(f"\nDiagnostics saved to {save_path}")

        plt.tight_layout()
        plt.show()

        # Additional separate plots
        self._plot_volatility_analysis(original_series, cv_results)
        self._plot_error_distribution(cv_results)

    def _plot_volatility_analysis(self, series, cv_results):
        """
        Detailed volatility analysis plots
        """
        fig, axes = plt.subplots(2, 2, figsize=(16, 10))

        # 1. Rolling Volatility
        returns = series.pct_change().dropna()
        rolling_vol_5 = returns.rolling(5).std()
        rolling_vol_20 = returns.rolling(20).std()
        rolling_vol_60 = returns.rolling(60).std()

        ax1 = axes[0, 0]
        ax1.plot(rolling_vol_5.index, rolling_vol_5, label='5-period', alpha=0.6)
        ax1.plot(rolling_vol_20.index, rolling_vol_20, label='20-period', linewidth=2)
        ax1.plot(rolling_vol_60.index, rolling_vol_60, label='60-period', linewidth=2)
        ax1.set_title('Rolling Volatility (Multiple Windows)', fontsize=12, fontweight='bold')
        ax1.set_ylabel('Volatility')
        ax1.legend()
        ax1.grid(True, alpha=0.3)

        # 2. Volatility Clustering
        ax2 = axes[0, 1]
        abs_returns = np.abs(returns)
        ax2.plot(abs_returns.index, abs_returns, alpha=0.5, linewidth=0.5)
        ax2.plot(abs_returns.rolling(20).mean().index,
                abs_returns.rolling(20).mean(),
                color='red', linewidth=2, label='20-period MA')
        ax2.set_title('Volatility Clustering (Absolute Returns)',
                     fontsize=12, fontweight='bold')
        ax2.set_ylabel('|Returns|')
        ax2.legend()
        ax2.grid(True, alpha=0.3)

        # 3. Squared Residuals ACF (ARCH test)
        ax3 = axes[1, 0]
        residuals = cv_results['predictions'] - cv_results['actuals']
        squared_residuals = residuals ** 2
        from statsmodels.graphics.tsaplots import plot_acf
        plot_acf(squared_residuals, lags=min(40, len(squared_residuals)//2),
                ax=ax3, alpha=0.05)
        ax3.set_title('ACF of Squared Residuals (ARCH Effects)',
                     fontsize=12, fontweight='bold')

        # 4. Volatility vs Prediction Error
        ax4 = axes[1, 1]
        # Estimate local volatility
        window = 20
        local_vol = pd.Series(cv_results['actuals']).rolling(window, min_periods=5).std()
        abs_errors = np.abs(residuals)

        ax4.scatter(local_vol, abs_errors, alpha=0.5, s=30)
        ax4.set_title('Prediction Error vs Local Volatility',
                     fontsize=12, fontweight='bold')
        ax4.set_xlabel('Local Volatility (20-period)')
        ax4.set_ylabel('Absolute Error')
        ax4.grid(True, alpha=0.3)

        # Add trend line
        valid_idx = ~np.isnan(local_vol)
        if valid_idx.sum() > 1:
            z = np.polyfit(local_vol[valid_idx], abs_errors[valid_idx], 1)
            p = np.poly1d(z)
            ax4.plot(local_vol[valid_idx], p(local_vol[valid_idx]),
                    "r--", linewidth=2, label=f'Trend: y={z[0]:.2f}x+{z[1]:.2f}')
            ax4.legend()

        plt.suptitle('Volatility Analysis', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()

    def _plot_error_distribution(self, cv_results):
        """
        Detailed error distribution analysis
        """
        fig, axes = plt.subplots(2, 3, figsize=(18, 10))

        residuals = cv_results['predictions'] - cv_results['actuals']
        abs_errors = np.abs(residuals)
        pct_errors = 100 * residuals / cv_results['actuals']

        # 1. Error over time
        ax1 = axes[0, 0]
        ax1.plot(residuals, alpha=0.7, linewidth=1)
        ax1.axhline(y=0, color='red', linestyle='--', linewidth=2)
        ax1.fill_between(range(len(residuals)), -2*residuals.std(),
                        2*residuals.std(), alpha=0.2, color='red')
        ax1.set_title('Residuals Over Time', fontsize=12, fontweight='bold')
        ax1.set_ylabel('Residual')
        ax1.grid(True, alpha=0.3)

        # 2. Absolute Error Distribution
        ax2 = axes[0, 1]
        ax2.hist(abs_errors, bins=30, edgecolor='black', alpha=0.7, color='orange')
        ax2.axvline(x=abs_errors.mean(), color='red', linestyle='--',
                   linewidth=2, label=f'Mean: {abs_errors.mean():.4f}')
        ax2.axvline(x=np.median(abs_errors), color='green', linestyle='--',
                   linewidth=2, label=f'Median: {np.median(abs_errors):.4f}')
        ax2.set_title('Absolute Error Distribution', fontsize=12, fontweight='bold')
        ax2.set_xlabel('Absolute Error')
        ax2.legend()
        ax2.grid(True, alpha=0.3)

        # 3. Percentage Error Distribution
        ax3 = axes[0, 2]
        ax3.hist(pct_errors, bins=30, edgecolor='black', alpha=0.7, color='purple')
        ax3.axvline(x=0, color='red', linestyle='--', linewidth=2)
        ax3.set_title('Percentage Error Distribution', fontsize=12, fontweight='bold')
        ax3.set_xlabel('Error (%)')
        ax3.grid(True, alpha=0.3)

        # 4. Error by Prediction Magnitude
        ax4 = axes[1, 0]
        pred_bins = pd.qcut(cv_results['predictions'], q=10, duplicates='drop')
        error_by_bin = pd.DataFrame({
            'pred_bin': pred_bins,
            'abs_error': abs_errors
        }).groupby('pred_bin')['abs_error'].agg(['mean', 'std'])

        x_pos = range(len(error_by_bin))
        ax4.bar(x_pos, error_by_bin['mean'], yerr=error_by_bin['std'],
               alpha=0.7, capsize=5)
        ax4.set_title('Error by Prediction Magnitude', fontsize=12, fontweight='bold')
        ax4.set_xlabel('Prediction Bin (Deciles)')
        ax4.set_ylabel('Mean Absolute Error')
        ax4.grid(True, alpha=0.3, axis='y')

        # 5. Cumulative Error
        ax5 = axes[1, 1]
        cumulative_error = np.cumsum(residuals)
        ax5.plot(cumulative_error, linewidth=2)
        ax5.axhline(y=0, color='red', linestyle='--', linewidth=2)
        ax5.set_title('Cumulative Error (Bias Detection)', fontsize=12, fontweight='bold')
        ax5.set_ylabel('Cumulative Error')
        ax5.grid(True, alpha=0.3)

        # 6. Error Metrics Summary
        ax6 = axes[1, 2]
        ax6.axis('off')

        metrics_text = f"""
        ERROR METRICS SUMMARY
        {'='*40}

        RMSE:        {np.sqrt(np.mean(residuals**2)):.6f}
        MAE:         {abs_errors.mean():.6f}
        Median AE:   {np.median(abs_errors):.6f}
        Max Error:   {abs_errors.max():.6f}

        Mean Error:  {residuals.mean():.6f}
        Std Error:   {residuals.std():.6f}

        MAPE:        {np.mean(np.abs(pct_errors)):.2f}%

        Error Range: [{residuals.min():.4f}, {residuals.max():.4f}]

        Errors > 2σ: {(abs_errors > 2*residuals.std()).sum()}
                     ({100*(abs_errors > 2*residuals.std()).sum()/len(residuals):.1f}%)
        """

        ax6.text(0.1, 0.9, metrics_text, transform=ax6.transAxes,
                fontsize=11, verticalalignment='top', family='monospace',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

        plt.suptitle('Error Distribution Analysis', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()

    def plot_feature_importance(self, features_df, top_n=20):
        """
        Plot feature importance from the fitted models
        """
        if not hasattr(self, 'mean_model') or self.mean_model is None:
            print("No model fitted yet. Run fit() first.")
            return

        # Get feature importance
        feature_cols = [col for col in features_df.columns if col != 'target']

        # Train a final model on all data to get feature importance
        X = features_df[feature_cols].values
        y = features_df['target'].values

        model = XGBRegressor(n_estimators=100, max_depth=4, random_state=42)
        model.fit(X, y, verbose=False)

        importance = model.feature_importances_
        feature_importance = pd.DataFrame({
            'feature': feature_cols,
            'importance': importance
        }).sort_values('importance', ascending=False)

        # Plot
        fig, axes = plt.subplots(1, 2, figsize=(16, 6))

        # Top N features
        top_features = feature_importance.head(top_n)
        ax1 = axes[0]
        ax1.barh(range(len(top_features)), top_features['importance'])
        ax1.set_yticks(range(len(top_features)))
        ax1.set_yticklabels(top_features['feature'])
        ax1.invert_yaxis()
        ax1.set_title(f'Top {top_n} Feature Importance', fontsize=12, fontweight='bold')
        ax1.set_xlabel('Importance Score')
        ax1.grid(True, alpha=0.3, axis='x')

        # Feature importance by category
        ax2 = axes[1]

        # Categorize features
        categories = {
            'Lag': feature_importance[feature_importance['feature'].str.contains('lag_')]['importance'].sum(),
            'Rolling Stats': feature_importance[feature_importance['feature'].str.contains('rolling_')]['importance'].sum(),
            'Volatility': feature_importance[feature_importance['feature'].str.contains('vol')]['importance'].sum(),
            'STL': feature_importance[feature_importance['feature'].str.contains('trend|seasonal')]['importance'].sum(),
            'Time': feature_importance[feature_importance['feature'].str.contains('day|month|quarter')]['importance'].sum()
        }

        colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']
        ax2.pie(categories.values(), labels=categories.keys(), autopct='%1.1f%%',
               colors=colors, startangle=90)
        ax2.set_title('Feature Importance by Category', fontsize=12, fontweight='bold')

        plt.tight_layout()
        plt.show()

        return feature_importance

    def fit(self, series, use_quantile=False):
        """
        Complete fitting pipeline
        """
        print("="*60)
        print("STARTING HETEROSCEDASTICITY-AWARE XGBOOST PIPELINE")
        print("="*60)

        # Step 1: Make stationary
        series_stationary = self.make_stationary(series)

        # Step 2: Find optimal lags
        self.find_optimal_lags(series_stationary)

        # Step 3: STL decomposition
        self.stl_decomposition(series_stationary)

        # Step 4: Create features
        features_df = self.create_features(series_stationary, include_volatility=True)

        # Step 5: Test heteroscedasticity (on a preliminary model)
        X_temp = features_df.drop('target', axis=1).values
        y_temp = features_df['target'].values

        temp_model = XGBRegressor(n_estimators=50, max_depth=3, random_state=42)
        temp_model.fit(X_temp[:len(X_temp)//2], y_temp[:len(y_temp)//2], verbose=False)
        temp_residuals = y_temp[:len(y_temp)//2] - temp_model.predict(X_temp[:len(X_temp)//2])

        is_hetero = self.test_heteroscedasticity(temp_residuals, X_temp[:len(X_temp)//2])

        if is_hetero:
            print("\n*** HETEROSCEDASTICITY DETECTED - Using volatility modeling ***")

        # Step 6: Rolling window cross-validation
        cv_results = self.rolling_window_cv(
            features_df,
            use_quantile=use_quantile,
            initial_train_size=self.seasonal_period
        )

        return cv_results


# Example usage
if __name__ == "__main__":
    # Generate synthetic commodity price data with heteroscedasticity
    np.random.seed(42)
    n = 500
    t = np.arange(n)

    # Trend + Seasonality + Heteroscedastic noise
    trend = 0.05 * t
    seasonal = 10 * np.sin(2 * np.pi * t / 12)

    # Heteroscedastic volatility (GARCH-like)
    volatility = np.zeros(n)
    volatility[0] = 1.0
    for i in range(1, n):
        volatility[i] = 0.1 + 0.85 * volatility[i-1] + 0.1 * np.random.randn()**2

    noise = volatility * np.random.randn(n)
    price = 100 + trend + seasonal + noise

    # Create time series
    dates = pd.date_range('2020-01-01', periods=n, freq='D')
    series = pd.Series(price, index=dates, name='price')

    print("Sample data shape:", series.shape)
    print("Sample data (first 5):", series.head())

    # Initialize and fit pipeline
    pipeline = HeteroscedasticXGBoostPipeline(seasonal_period=12, max_lags=20)

    # Option 1: Quantile regression (recommended for heteroscedasticity)
    results_quantile = pipeline.fit(series, use_quantile=True)

    # Option 2: Two-model approach (mean + volatility)
    # results_two_model = pipeline.fit(series, use_quantile=False)

    print("\n" + "="*60)
    print("PIPELINE COMPLETE")
    print("="*60)

    # Visualize diagnostics
    pipeline.plot_diagnostics(series, results_quantile)

Sample data shape: (500,)
Sample data (first 5): 2020-01-01    98.6172
2020-01-02   105.9527
2020-01-03   110.5367
2020-01-04   108.8454
2020-01-05   109.4935
Freq: D, Name: price, dtype: float64
STARTING HETEROSCEDASTICITY-AWARE XGBOOST PIPELINE

=== STATIONARITY TESTS ===

Augmented Dickey-Fuller Test:
  ADF Statistic: -0.5331
  p-value: 0.8854
  Critical Values: {'1%': np.float64(-3.4438771098680196), '5%': np.float64(-2.867505393939065), '10%': np.float64(-2.569947324764179)}
  Result: NON-STATIONARY

KPSS Test:
  KPSS Statistic: 0.0220
  p-value: 0.1000
  Critical Values: {'10%': 0.119, '5%': 0.146, '2.5%': 0.176, '1%': 0.216}
  Result: STATIONARY

=== MAKING SERIES STATIONARY ===

=== STATIONARITY TESTS ===

Augmented Dickey-Fuller Test:
  ADF Statistic: -0.9713
  p-value: 0.7636
  Critical Values: {'1%': np.float64(-3.4438771098680196), '5%': np.float64(-2.867505393939065), '10%': np.float64(-2.569947324764179)}
  Result: NON-STATIONARY

KPSS Test:
  KPSS Statistic: 0.0413
  p-v

KeyboardInterrupt: 

In [ ]:
"""
Heteroscedasticity-Aware XGBoost Pipeline for Commodity Price Prediction
Includes: Stationarity testing, STL decomposition, volatility modeling, rolling window CV
"""

import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from statsmodels.tsa.stattools import adfuller, kpss, acf, pacf
from statsmodels.tsa.seasonal import STL
from statsmodels.stats.diagnostic import het_breuschpagan, het_white
from scipy.stats import boxcox, jarque_bera
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
import seaborn as sns

class HeteroscedasticXGBoostPipeline:
    """
    Complete pipeline for commodity price prediction with heteroscedasticity handling
    """

    def __init__(self, seasonal_period=12, max_lags=20):
        """
        Parameters:
        -----------
        seasonal_period : int
            Expected seasonality period (12 for monthly, 252 for daily trading days)
            Note: Will be adjusted to odd number >= 3 for STL decomposition
        max_lags : int
            Maximum number of lags to consider
        """
        self.seasonal_period = seasonal_period
        self.max_lags = max_lags
        self.differencing_order = 0
        self.transformation = None
        self.scaler = StandardScaler()
        self.optimal_lags = []

        # Models
        self.mean_model = None
        self.vol_model = None
        self.quantile_models = {}

        # Results storage
        self.stationarity_results = {}
        self.heteroscedasticity_results = {}
        self.stl_components = None

    def test_stationarity(self, series, alpha=0.05):
        """
        Perform ADF and KPSS tests for stationarity
        """
        print("\n=== STATIONARITY TESTS ===")

        # ADF Test (H0: unit root present - non-stationary)
        adf_result = adfuller(series.dropna(), autolag='AIC')
        print(f"\nAugmented Dickey-Fuller Test:")
        print(f"  ADF Statistic: {adf_result[0]:.4f}")
        print(f"  p-value: {adf_result[1]:.4f}")
        print(f"  Critical Values: {adf_result[4]}")
        adf_stationary = adf_result[1] < alpha
        print(f"  Result: {'STATIONARY' if adf_stationary else 'NON-STATIONARY'}")

        # KPSS Test (H0: series is stationary)
        kpss_result = kpss(series.dropna(), regression='ct', nlags='auto')
        print(f"\nKPSS Test:")
        print(f"  KPSS Statistic: {kpss_result[0]:.4f}")
        print(f"  p-value: {kpss_result[1]:.4f}")
        print(f"  Critical Values: {kpss_result[3]}")
        kpss_stationary = kpss_result[1] > alpha
        print(f"  Result: {'STATIONARY' if kpss_stationary else 'NON-STATIONARY'}")

        self.stationarity_results = {
            'adf_statistic': adf_result[0],
            'adf_pvalue': adf_result[1],
            'adf_stationary': adf_stationary,
            'kpss_statistic': kpss_result[0],
            'kpss_pvalue': kpss_result[1],
            'kpss_stationary': kpss_stationary,
            'is_stationary': adf_stationary and kpss_stationary
        }

        return self.stationarity_results['is_stationary']

    def make_stationary(self, series):
        """
        Apply transformations to make series stationary
        """
        original_series = series.copy()

        if not self.test_stationarity(series):
            print("\n=== MAKING SERIES STATIONARY ===")

            # Try log transformation first (variance stabilization)
            if (series > 0).all():
                series_log = np.log(series)
                if self.test_stationarity(series_log):
                    print("Log transformation sufficient")
                    self.transformation = 'log'
                    return series_log

            # Apply first differencing
            series_diff = series.diff().dropna()
            if self.test_stationarity(series_diff):
                print("First differencing sufficient")
                self.differencing_order = 1
                self.transformation = 'diff'
                return series_diff

            # Apply second differencing if needed
            series_diff2 = series_diff.diff().dropna()
            if self.test_stationarity(series_diff2):
                print("Second differencing required")
                self.differencing_order = 2
                self.transformation = 'diff2'
                return series_diff2

            print("Warning: Could not achieve stationarity. Proceeding with first difference.")
            self.differencing_order = 1
            self.transformation = 'diff'
            return series.diff().dropna()

        print("Series is already stationary")
        self.transformation = 'none'
        return series

    def find_optimal_lags(self, series, method='acf', threshold=0.2):
        """
        Determine optimal lag order using ACF/PACF
        """
        print("\n=== FINDING OPTIMAL LAGS ===")

        series_clean = series.dropna()

        # ACF
        acf_values = acf(series_clean, nlags=self.max_lags, fft=False)
        significant_acf = np.where(np.abs(acf_values[1:]) > threshold)[0] + 1

        # PACF
        pacf_values = pacf(series_clean, nlags=self.max_lags, method='ols')
        significant_pacf = np.where(np.abs(pacf_values[1:]) > threshold)[0] + 1

        if method == 'acf':
            self.optimal_lags = significant_acf[:10].tolist()  # Top 10 lags
        elif method == 'pacf':
            self.optimal_lags = significant_pacf[:10].tolist()
        else:  # both
            self.optimal_lags = sorted(list(set(significant_acf[:10].tolist() +
                                                 significant_pacf[:10].tolist())))

        if not self.optimal_lags:
            self.optimal_lags = [1, 2, 3, 5, 7]  # Default lags

        print(f"Optimal lags: {self.optimal_lags}")
        return self.optimal_lags

    def stl_decomposition(self, series):
        """
        Perform STL decomposition
        """
        print("\n=== STL DECOMPOSITION ===")

        series_clean = series.dropna()

        if len(series_clean) < 2 * self.seasonal_period:
            print(f"Warning: Series too short for seasonal decomposition (need >= {2*self.seasonal_period})")
            return None

        # Ensure seasonal parameter is odd and >= 3
        seasonal_param = self.seasonal_period
        if seasonal_param % 2 == 0:
            seasonal_param += 1
        if seasonal_param < 3:
            seasonal_param = 3

        print(f"Using seasonal parameter: {seasonal_param}")

        try:
            stl = STL(series_clean, seasonal=seasonal_param, robust=True)
            result = stl.fit()
        except Exception as e:
            print(f"STL decomposition failed: {e}")
            print("Proceeding without STL components")
            return None

        self.stl_components = {
            'trend': result.trend,
            'seasonal': result.seasonal,
            'resid': result.resid
        }

        print(f"Trend strength: {1 - result.resid.var() / (result.trend + result.resid).var():.4f}")
        print(f"Seasonal strength: {1 - result.resid.var() / (result.seasonal + result.resid).var():.4f}")

        return self.stl_components

    def test_heteroscedasticity(self, residuals, features):
        """
        Test for heteroscedasticity using Breusch-Pagan and White tests
        """
        print("\n=== HETEROSCEDASTICITY TESTS ===")

        # Breusch-Pagan test
        try:
            bp_stat, bp_pval, _, _ = het_breuschpagan(residuals, features)
            print(f"\nBreusch-Pagan Test:")
            print(f"  LM Statistic: {bp_stat:.4f}")
            print(f"  p-value: {bp_pval:.4f}")
            bp_hetero = bp_pval < 0.05
            print(f"  Result: {'HETEROSCEDASTIC' if bp_hetero else 'HOMOSCEDASTIC'}")
        except:
            bp_hetero = True
            bp_pval = 0.0
            print("Breusch-Pagan test failed")

        # White test
        try:
            white_stat, white_pval, _, _ = het_white(residuals, features)
            print(f"\nWhite Test:")
            print(f"  LM Statistic: {white_stat:.4f}")
            print(f"  p-value: {white_pval:.4f}")
            white_hetero = white_pval < 0.05
            print(f"  Result: {'HETEROSCEDASTIC' if white_hetero else 'HOMOSCEDASTIC'}")
        except:
            white_hetero = True
            white_pval = 0.0
            print("White test failed")

        self.heteroscedasticity_results = {
            'bp_pvalue': bp_pval,
            'white_pvalue': white_pval,
            'is_heteroscedastic': bp_hetero or white_hetero
        }

        return self.heteroscedasticity_results['is_heteroscedastic']

    def create_features(self, series, include_volatility=True):
        """
        Create feature matrix with lags, rolling statistics, and volatility features
        """
        df = pd.DataFrame({'target': series})

        # Lag features
        for lag in self.optimal_lags:
            df[f'lag_{lag}'] = df['target'].shift(lag)

        # Rolling statistics
        for window in [5, 10, 20]:
            df[f'rolling_mean_{window}'] = df['target'].rolling(window).mean()
            df[f'rolling_std_{window}'] = df['target'].rolling(window).std()
            df[f'rolling_min_{window}'] = df['target'].rolling(window).min()
            df[f'rolling_max_{window}'] = df['target'].rolling(window).max()

        # Volatility features (if enabled)
        if include_volatility:
            # Historical volatility at multiple horizons
            for window in [5, 10, 20, 60]:
                df[f'volatility_{window}'] = df['target'].rolling(window).std()

            # Realized volatility (if returns)
            df['realized_vol'] = df['target'].rolling(20).std() * np.sqrt(252)

            # Volatility of volatility
            df['vol_of_vol'] = df['volatility_20'].rolling(20).std()

            # Volatility regime (high/low)
            vol_median = df['volatility_20'].rolling(60).median()
            df['high_vol_regime'] = (df['volatility_20'] > vol_median).astype(int)

            # Range-based volatility proxy
            df['range_vol'] = (df['rolling_max_20'] - df['rolling_min_20']) / df['rolling_mean_20']

        # STL components (if available)
        if self.stl_components is not None:
            df['trend'] = self.stl_components['trend']
            df['seasonal'] = self.stl_components['seasonal']

        # Time features
        if isinstance(df.index, pd.DatetimeIndex):
            df['day_of_week'] = df.index.dayofweek
            df['month'] = df.index.month
            df['quarter'] = df.index.quarter
            df['day_of_month'] = df.index.day

        # Drop NaN rows
        df = df.dropna()

        return df

    def create_volatility_weights(self, series, window=20, method='inverse_variance'):
        """
        Create sample weights based on volatility (optimized version)
        """
        # Convert to numpy array if pandas Series
        if isinstance(series, pd.Series):
            values = series.values
        else:
            values = np.array(series)

        if method == 'inverse_variance':
            # Use pandas rolling for speed (with proper handling)
            series_pd = pd.Series(values)
            rolling_var = series_pd.rolling(window, min_periods=1).var()

            # Clean up variance
            rolling_var = rolling_var.fillna(rolling_var.median())
            rolling_var = np.clip(rolling_var, 1e-10, None)  # Floor at small positive

            # Inverse variance weights
            weights = 1.0 / rolling_var.values

        elif method == 'exponential':
            half_life = window
            weights = np.exp(-np.log(2) * np.arange(len(values))[::-1] / half_life)
        else:
            weights = np.ones(len(values))

        # Fast cleanup
        weights = np.clip(weights, 1e-8, 1e8)  # Clip to reasonable range
        weights[~np.isfinite(weights)] = 1.0  # Replace NaN/Inf

        # Normalize
        weights = weights / np.sum(weights) * len(weights)

        return weights

    def rolling_window_cv(self, data, target_col='target',
                          initial_train_size=None, step_size=1,
                          use_quantile=False, quantiles=[0.1, 0.5, 0.9],
                          n_estimators=100, max_depth=4, verbose=True):
        """
        Perform rolling window cross-validation (optimized)

        Parameters:
        -----------
        data : DataFrame
            Feature matrix with target column
        target_col : str
            Name of target column
        initial_train_size : int or None
            Initial training window size (if None, uses one seasonal period)
        step_size : int
            Number of periods to roll forward
        use_quantile : bool
            Whether to use quantile regression
        quantiles : list
            Quantiles to predict if use_quantile=True
        n_estimators : int
            Number of trees (lower = faster)
        max_depth : int
            Tree depth (lower = faster)
        verbose : bool
            Print progress
        """
        if verbose:
            print("\n=== ROLLING WINDOW CROSS-VALIDATION ===")

        if initial_train_size is None:
            initial_train_size = self.seasonal_period

        feature_cols = [col for col in data.columns if col != target_col]
        X = data[feature_cols].values
        y = data[target_col].values

        predictions = []
        actuals = []
        prediction_intervals = []
        fold_metrics = []

        n_folds = (len(data) - initial_train_size) // step_size

        for fold in range(n_folds):
            train_end = initial_train_size + fold * step_size
            test_start = train_end
            test_end = min(test_start + step_size, len(data))

            if test_end >= len(data):
                break

            X_train, y_train = X[:train_end], y[:train_end]
            X_test, y_test = X[test_start:test_end], y[test_start:test_end]

            if verbose:
                print(f"\nFold {fold + 1}/{n_folds}: Train={len(X_train)}, Test={len(X_test)}")

            # Simplified: Use uniform weights for speed (comment out for weighted version)
            # weights = np.ones(len(y_train))

            # Or use exponential weights (faster than inverse variance)
            weights = self.create_volatility_weights(
                y_train,
                window=min(20, len(y_train)//2),
                method='exponential'  # Much faster than inverse_variance
            )

            if use_quantile:
                # Quantile regression approach
                fold_predictions = {}

                for q in quantiles:
                    model = XGBRegressor(
                        objective='reg:quantileerror',
                        quantile_alpha=q,
                        n_estimators=n_estimators,  # Reduced from 100
                        max_depth=max_depth,        # Reduced from 4
                        learning_rate=0.1,          # Increased from 0.05 for speed
                        subsample=0.8,
                        colsample_bytree=0.8,
                        n_jobs=-1,                  # Use all cores
                        random_state=42,
                        tree_method='hist'          # Faster tree building
                    )
                    model.fit(X_train, y_train, sample_weight=weights, verbose=0)
                    fold_predictions[q] = model.predict(X_test)

                # Use median as point prediction
                pred = fold_predictions[0.5]
                lower = fold_predictions[quantiles[0]]
                upper = fold_predictions[quantiles[-1]]

                prediction_intervals.extend(list(zip(lower, upper)))

            else:
                # Two-model approach: Mean + Volatility

                # Model 1: Mean prediction
                mean_model = XGBRegressor(
                    n_estimators=n_estimators,
                    max_depth=max_depth,
                    learning_rate=0.1,
                    subsample=0.8,
                    colsample_bytree=0.8,
                    n_jobs=-1,
                    random_state=42,
                    tree_method='hist'
                )
                mean_model.fit(X_train, y_train, sample_weight=weights, verbose=0)
                pred_mean = mean_model.predict(X_test)

                # Model 2: Volatility prediction (simplified)
                train_residuals = y_train - mean_model.predict(X_train)
                train_vol = pd.Series(train_residuals).rolling(20, min_periods=5).std().fillna(
                    pd.Series(train_residuals).std()
                ).values

                vol_model = XGBRegressor(
                    n_estimators=n_estimators//2,  # Half the trees
                    max_depth=3,
                    learning_rate=0.1,
                    subsample=0.8,
                    colsample_bytree=0.8,
                    n_jobs=-1,
                    random_state=42,
                    tree_method='hist'
                )
                vol_model.fit(X_train, train_vol, verbose=0)
                pred_vol = vol_model.predict(X_test)

                pred = pred_mean
                lower = pred_mean - 1.96 * pred_vol
                upper = pred_mean + 1.96 * pred_vol

                prediction_intervals.extend(list(zip(lower, upper)))

            predictions.extend(pred)
            actuals.extend(y_test)

            # Calculate fold metrics
            rmse = np.sqrt(np.mean((pred - y_test) ** 2))
            mae = np.mean(np.abs(pred - y_test))

            # Coverage of prediction intervals
            in_interval = np.sum((y_test >= lower) & (y_test <= upper))
            coverage = in_interval / len(y_test)

            fold_metrics.append({
                'fold': fold + 1,
                'rmse': rmse,
                'mae': mae,
                'coverage': coverage
            })

            if verbose:
                print(f"  RMSE: {rmse:.4f}, MAE: {mae:.4f}, Coverage: {coverage:.2%}")

        results = {
            'predictions': np.array(predictions),
            'actuals': np.array(actuals),
            'prediction_intervals': prediction_intervals,
            'fold_metrics': pd.DataFrame(fold_metrics)
        }

        # Overall metrics
        overall_rmse = np.sqrt(np.mean((results['predictions'] - results['actuals']) ** 2))
        overall_mae = np.mean(np.abs(results['predictions'] - results['actuals']))

        if verbose:
            print(f"\n=== OVERALL CROSS-VALIDATION RESULTS ===")
            print(f"RMSE: {overall_rmse:.4f}")
            print(f"MAE: {overall_mae:.4f}")
            print(f"Mean Coverage: {results['fold_metrics']['coverage'].mean():.2%}")

        return results

    def plot_diagnostics(self, original_series, cv_results, save_path=None):
        """
        Comprehensive diagnostic plots for model evaluation

        Parameters:
        -----------
        original_series : pd.Series
            Original time series data
        cv_results : dict
            Results from rolling_window_cv
        save_path : str, optional
            Path to save figures
        """
        fig = plt.figure(figsize=(20, 16))
        gs = fig.add_gridspec(5, 3, hspace=0.3, wspace=0.3)

        # 1. Original Series with Trend and Seasonality
        ax1 = fig.add_subplot(gs[0, :])
        ax1.plot(original_series.index, original_series.values,
                 label='Original', alpha=0.7, linewidth=1)
        if self.stl_components is not None:
            ax1.plot(self.stl_components['trend'].index,
                    self.stl_components['trend'].values,
                    label='Trend', linewidth=2, color='red')
        ax1.set_title('Original Time Series with Trend', fontsize=14, fontweight='bold')
        ax1.set_xlabel('Time')
        ax1.set_ylabel('Price')
        ax1.legend()
        ax1.grid(True, alpha=0.3)

        # 2. STL Decomposition Components
        if self.stl_components is not None:
            ax2 = fig.add_subplot(gs[1, 0])
            ax2.plot(self.stl_components['trend'].index,
                    self.stl_components['trend'].values, color='red')
            ax2.set_title('Trend Component', fontsize=12, fontweight='bold')
            ax2.grid(True, alpha=0.3)

            ax3 = fig.add_subplot(gs[1, 1])
            ax3.plot(self.stl_components['seasonal'].index,
                    self.stl_components['seasonal'].values, color='green')
            ax3.set_title('Seasonal Component', fontsize=12, fontweight='bold')
            ax3.grid(True, alpha=0.3)

            ax4 = fig.add_subplot(gs[1, 2])
            ax4.plot(self.stl_components['resid'].index,
                    self.stl_components['resid'].values, color='purple', alpha=0.6)
            ax4.set_title('Residual Component', fontsize=12, fontweight='bold')
            ax4.grid(True, alpha=0.3)

        # 3. Predictions vs Actuals
        ax5 = fig.add_subplot(gs[2, :])
        test_indices = range(len(cv_results['actuals']))
        ax5.plot(test_indices, cv_results['actuals'],
                label='Actual', color='blue', linewidth=2, alpha=0.7)
        ax5.plot(test_indices, cv_results['predictions'],
                label='Predicted', color='red', linewidth=2, alpha=0.7)

        # Add prediction intervals
        if cv_results['prediction_intervals']:
            intervals = np.array(cv_results['prediction_intervals'])
            ax5.fill_between(test_indices, intervals[:, 0], intervals[:, 1],
                           alpha=0.2, color='red', label='95% Prediction Interval')

        ax5.set_title('Predictions vs Actuals (Cross-Validation)',
                     fontsize=14, fontweight='bold')
        ax5.set_xlabel('Test Sample Index')
        ax5.set_ylabel('Value')
        ax5.legend()
        ax5.grid(True, alpha=0.3)

        # 4. Residuals Analysis
        residuals = cv_results['predictions'] - cv_results['actuals']

        ax6 = fig.add_subplot(gs[3, 0])
        ax6.scatter(cv_results['predictions'], residuals, alpha=0.5, s=20)
        ax6.axhline(y=0, color='red', linestyle='--', linewidth=2)
        ax6.set_title('Residual Plot', fontsize=12, fontweight='bold')
        ax6.set_xlabel('Predicted Values')
        ax6.set_ylabel('Residuals')
        ax6.grid(True, alpha=0.3)

        # Add LOESS smooth to detect patterns
        from scipy.signal import savgol_filter
        if len(residuals) > 50:
            sorted_idx = np.argsort(cv_results['predictions'])
            smoothed = savgol_filter(residuals[sorted_idx],
                                    min(51, len(residuals)//2*2-1), 3)
            ax6.plot(cv_results['predictions'][sorted_idx], smoothed,
                    color='orange', linewidth=2, label='Smoothed')
            ax6.legend()

        # 5. Residuals Histogram
        ax7 = fig.add_subplot(gs[3, 1])
        ax7.hist(residuals, bins=30, edgecolor='black', alpha=0.7)
        ax7.axvline(x=0, color='red', linestyle='--', linewidth=2)
        ax7.set_title('Residuals Distribution', fontsize=12, fontweight='bold')
        ax7.set_xlabel('Residual Value')
        ax7.set_ylabel('Frequency')

        # Add normality statistics
        from scipy.stats import shapiro
        _, p_value = shapiro(residuals)
        ax7.text(0.05, 0.95, f'Shapiro-Wilk p={p_value:.4f}',
                transform=ax7.transAxes, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
        ax7.grid(True, alpha=0.3)

        # 6. Q-Q Plot
        ax8 = fig.add_subplot(gs[3, 2])
        from scipy.stats import probplot
        probplot(residuals, dist="norm", plot=ax8)
        ax8.set_title('Q-Q Plot', fontsize=12, fontweight='bold')
        ax8.grid(True, alpha=0.3)

        # 7. ACF of Residuals
        ax9 = fig.add_subplot(gs[4, 0])
        from statsmodels.graphics.tsaplots import plot_acf
        plot_acf(residuals, lags=min(40, len(residuals)//2), ax=ax9, alpha=0.05)
        ax9.set_title('ACF of Residuals', fontsize=12, fontweight='bold')
        ax9.set_xlabel('Lag')

        # 8. Rolling RMSE
        ax10 = fig.add_subplot(gs[4, 1])
        fold_metrics = cv_results['fold_metrics']
        ax10.plot(fold_metrics['fold'], fold_metrics['rmse'],
                 marker='o', linewidth=2, markersize=8)
        ax10.set_title('RMSE by Fold', fontsize=12, fontweight='bold')
        ax10.set_xlabel('Fold')
        ax10.set_ylabel('RMSE')
        ax10.grid(True, alpha=0.3)

        # 9. Coverage Analysis
        ax11 = fig.add_subplot(gs[4, 2])
        ax11.plot(fold_metrics['fold'], fold_metrics['coverage'],
                 marker='s', linewidth=2, markersize=8, color='green')
        ax11.axhline(y=0.95, color='red', linestyle='--', linewidth=2,
                    label='Target 95%')
        ax11.set_title('Prediction Interval Coverage', fontsize=12, fontweight='bold')
        ax11.set_xlabel('Fold')
        ax11.set_ylabel('Coverage')
        ax11.set_ylim([0, 1])
        ax11.legend()
        ax11.grid(True, alpha=0.3)

        plt.suptitle('Comprehensive Model Diagnostics',
                    fontsize=16, fontweight='bold', y=0.995)

        if save_path:
            plt.savefig(save_path, dpi=300, bbox_inches='tight')
            print(f"\nDiagnostics saved to {save_path}")

        plt.tight_layout()
        plt.show()

        # Additional separate plots
        self._plot_volatility_analysis(original_series, cv_results)
        self._plot_error_distribution(cv_results)

    def _plot_volatility_analysis(self, series, cv_results):
        """
        Detailed volatility analysis plots
        """
        fig, axes = plt.subplots(2, 2, figsize=(16, 10))

        # 1. Rolling Volatility
        returns = series.pct_change().dropna()
        rolling_vol_5 = returns.rolling(5).std()
        rolling_vol_20 = returns.rolling(20).std()
        rolling_vol_60 = returns.rolling(60).std()

        ax1 = axes[0, 0]
        ax1.plot(rolling_vol_5.index, rolling_vol_5, label='5-period', alpha=0.6)
        ax1.plot(rolling_vol_20.index, rolling_vol_20, label='20-period', linewidth=2)
        ax1.plot(rolling_vol_60.index, rolling_vol_60, label='60-period', linewidth=2)
        ax1.set_title('Rolling Volatility (Multiple Windows)', fontsize=12, fontweight='bold')
        ax1.set_ylabel('Volatility')
        ax1.legend()
        ax1.grid(True, alpha=0.3)

        # 2. Volatility Clustering
        ax2 = axes[0, 1]
        abs_returns = np.abs(returns)
        ax2.plot(abs_returns.index, abs_returns, alpha=0.5, linewidth=0.5)
        ax2.plot(abs_returns.rolling(20).mean().index,
                abs_returns.rolling(20).mean(),
                color='red', linewidth=2, label='20-period MA')
        ax2.set_title('Volatility Clustering (Absolute Returns)',
                     fontsize=12, fontweight='bold')
        ax2.set_ylabel('|Returns|')
        ax2.legend()
        ax2.grid(True, alpha=0.3)

        # 3. Squared Residuals ACF (ARCH test)
        ax3 = axes[1, 0]
        residuals = cv_results['predictions'] - cv_results['actuals']
        squared_residuals = residuals ** 2
        from statsmodels.graphics.tsaplots import plot_acf
        plot_acf(squared_residuals, lags=min(40, len(squared_residuals)//2),
                ax=ax3, alpha=0.05)
        ax3.set_title('ACF of Squared Residuals (ARCH Effects)',
                     fontsize=12, fontweight='bold')

        # 4. Volatility vs Prediction Error
        ax4 = axes[1, 1]
        # Estimate local volatility
        window = 20
        local_vol = pd.Series(cv_results['actuals']).rolling(window, min_periods=5).std()
        abs_errors = np.abs(residuals)

        ax4.scatter(local_vol, abs_errors, alpha=0.5, s=30)
        ax4.set_title('Prediction Error vs Local Volatility',
                     fontsize=12, fontweight='bold')
        ax4.set_xlabel('Local Volatility (20-period)')
        ax4.set_ylabel('Absolute Error')
        ax4.grid(True, alpha=0.3)

        # Add trend line
        valid_idx = ~np.isnan(local_vol)
        if valid_idx.sum() > 1:
            z = np.polyfit(local_vol[valid_idx], abs_errors[valid_idx], 1)
            p = np.poly1d(z)
            ax4.plot(local_vol[valid_idx], p(local_vol[valid_idx]),
                    "r--", linewidth=2, label=f'Trend: y={z[0]:.2f}x+{z[1]:.2f}')
            ax4.legend()

        plt.suptitle('Volatility Analysis', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()

    def _plot_error_distribution(self, cv_results):
        """
        Detailed error distribution analysis
        """
        fig, axes = plt.subplots(2, 3, figsize=(18, 10))

        residuals = cv_results['predictions'] - cv_results['actuals']
        abs_errors = np.abs(residuals)
        pct_errors = 100 * residuals / cv_results['actuals']

        # 1. Error over time
        ax1 = axes[0, 0]
        ax1.plot(residuals, alpha=0.7, linewidth=1)
        ax1.axhline(y=0, color='red', linestyle='--', linewidth=2)
        ax1.fill_between(range(len(residuals)), -2*residuals.std(),
                        2*residuals.std(), alpha=0.2, color='red')
        ax1.set_title('Residuals Over Time', fontsize=12, fontweight='bold')
        ax1.set_ylabel('Residual')
        ax1.grid(True, alpha=0.3)

        # 2. Absolute Error Distribution
        ax2 = axes[0, 1]
        ax2.hist(abs_errors, bins=30, edgecolor='black', alpha=0.7, color='orange')
        ax2.axvline(x=abs_errors.mean(), color='red', linestyle='--',
                   linewidth=2, label=f'Mean: {abs_errors.mean():.4f}')
        ax2.axvline(x=np.median(abs_errors), color='green', linestyle='--',
                   linewidth=2, label=f'Median: {np.median(abs_errors):.4f}')
        ax2.set_title('Absolute Error Distribution', fontsize=12, fontweight='bold')
        ax2.set_xlabel('Absolute Error')
        ax2.legend()
        ax2.grid(True, alpha=0.3)

        # 3. Percentage Error Distribution
        ax3 = axes[0, 2]
        ax3.hist(pct_errors, bins=30, edgecolor='black', alpha=0.7, color='purple')
        ax3.axvline(x=0, color='red', linestyle='--', linewidth=2)
        ax3.set_title('Percentage Error Distribution', fontsize=12, fontweight='bold')
        ax3.set_xlabel('Error (%)')
        ax3.grid(True, alpha=0.3)

        # 4. Error by Prediction Magnitude
        ax4 = axes[1, 0]
        pred_bins = pd.qcut(cv_results['predictions'], q=10, duplicates='drop')
        error_by_bin = pd.DataFrame({
            'pred_bin': pred_bins,
            'abs_error': abs_errors
        }).groupby('pred_bin')['abs_error'].agg(['mean', 'std'])

        x_pos = range(len(error_by_bin))
        ax4.bar(x_pos, error_by_bin['mean'], yerr=error_by_bin['std'],
               alpha=0.7, capsize=5)
        ax4.set_title('Error by Prediction Magnitude', fontsize=12, fontweight='bold')
        ax4.set_xlabel('Prediction Bin (Deciles)')
        ax4.set_ylabel('Mean Absolute Error')
        ax4.grid(True, alpha=0.3, axis='y')

        # 5. Cumulative Error
        ax5 = axes[1, 1]
        cumulative_error = np.cumsum(residuals)
        ax5.plot(cumulative_error, linewidth=2)
        ax5.axhline(y=0, color='red', linestyle='--', linewidth=2)
        ax5.set_title('Cumulative Error (Bias Detection)', fontsize=12, fontweight='bold')
        ax5.set_ylabel('Cumulative Error')
        ax5.grid(True, alpha=0.3)

        # 6. Error Metrics Summary
        ax6 = axes[1, 2]
        ax6.axis('off')

        metrics_text = f"""
        ERROR METRICS SUMMARY
        {'='*40}

        RMSE:        {np.sqrt(np.mean(residuals**2)):.6f}
        MAE:         {abs_errors.mean():.6f}
        Median AE:   {np.median(abs_errors):.6f}
        Max Error:   {abs_errors.max():.6f}

        Mean Error:  {residuals.mean():.6f}
        Std Error:   {residuals.std():.6f}

        MAPE:        {np.mean(np.abs(pct_errors)):.2f}%

        Error Range: [{residuals.min():.4f}, {residuals.max():.4f}]

        Errors > 2σ: {(abs_errors > 2*residuals.std()).sum()}
                     ({100*(abs_errors > 2*residuals.std()).sum()/len(residuals):.1f}%)
        """

        ax6.text(0.1, 0.9, metrics_text, transform=ax6.transAxes,
                fontsize=11, verticalalignment='top', family='monospace',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

        plt.suptitle('Error Distribution Analysis', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()

    def plot_feature_importance(self, features_df, top_n=20):
        """
        Plot feature importance from the fitted models
        """
        if not hasattr(self, 'mean_model') or self.mean_model is None:
            print("No model fitted yet. Run fit() first.")
            return

        # Get feature importance
        feature_cols = [col for col in features_df.columns if col != 'target']

        # Train a final model on all data to get feature importance
        X = features_df[feature_cols].values
        y = features_df['target'].values

        model = XGBRegressor(n_estimators=100, max_depth=4, random_state=42)
        model.fit(X, y, verbose=False)

        importance = model.feature_importances_
        feature_importance = pd.DataFrame({
            'feature': feature_cols,
            'importance': importance
        }).sort_values('importance', ascending=False)

        # Plot
        fig, axes = plt.subplots(1, 2, figsize=(16, 6))

        # Top N features
        top_features = feature_importance.head(top_n)
        ax1 = axes[0]
        ax1.barh(range(len(top_features)), top_features['importance'])
        ax1.set_yticks(range(len(top_features)))
        ax1.set_yticklabels(top_features['feature'])
        ax1.invert_yaxis()
        ax1.set_title(f'Top {top_n} Feature Importance', fontsize=12, fontweight='bold')
        ax1.set_xlabel('Importance Score')
        ax1.grid(True, alpha=0.3, axis='x')

        # Feature importance by category
        ax2 = axes[1]

        # Categorize features
        categories = {
            'Lag': feature_importance[feature_importance['feature'].str.contains('lag_')]['importance'].sum(),
            'Rolling Stats': feature_importance[feature_importance['feature'].str.contains('rolling_')]['importance'].sum(),
            'Volatility': feature_importance[feature_importance['feature'].str.contains('vol')]['importance'].sum(),
            'STL': feature_importance[feature_importance['feature'].str.contains('trend|seasonal')]['importance'].sum(),
            'Time': feature_importance[feature_importance['feature'].str.contains('day|month|quarter')]['importance'].sum()
        }

        colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']
        ax2.pie(categories.values(), labels=categories.keys(), autopct='%1.1f%%',
               colors=colors, startangle=90)
        ax2.set_title('Feature Importance by Category', fontsize=12, fontweight='bold')

        plt.tight_layout()
        plt.show()

        return feature_importance

    def fit(self, series, use_quantile=False, fast_mode=True):
        """
        Complete fitting pipeline

        Parameters:
        -----------
        series : pd.Series
            Time series data
        use_quantile : bool
            Use quantile regression (slower but better intervals)
        fast_mode : bool
            Use faster settings (fewer trees, larger steps)
        """
        print("="*60)
        print("STARTING HETEROSCEDASTICITY-AWARE XGBOOST PIPELINE")
        print("="*60)

        # Step 1: Make stationary
        series_stationary = self.make_stationary(series)

        # Step 2: Find optimal lags
        self.find_optimal_lags(series_stationary)

        # Step 3: STL decomposition
        self.stl_decomposition(series_stationary)

        # Step 4: Create features
        features_df = self.create_features(series_stationary, include_volatility=True)

        # Step 5: Test heteroscedasticity (skip in fast mode)
        if not fast_mode:
            X_temp = features_df.drop('target', axis=1).values
            y_temp = features_df['target'].values

            temp_model = XGBRegressor(n_estimators=50, max_depth=3, random_state=42, n_jobs=-1)
            temp_model.fit(X_temp[:len(X_temp)//2], y_temp[:len(y_temp)//2], verbose=False)
            temp_residuals = y_temp[:len(y_temp)//2] - temp_model.predict(X_temp[:len(X_temp)//2])

            is_hetero = self.test_heteroscedasticity(temp_residuals, X_temp[:len(X_temp)//2])

            if is_hetero:
                print("\n*** HETEROSCEDASTICITY DETECTED - Using volatility modeling ***")

        # Step 6: Rolling window cross-validation (optimized)
        if fast_mode:
            n_estimators = 50    # Reduced trees
            step_size = max(1, len(features_df) // 20)  # Larger steps = fewer folds
            print(f"\nFast mode: {n_estimators} trees, step_size={step_size}")
        else:
            n_estimators = 100
            step_size = 1

        cv_results = self.rolling_window_cv(
            features_df,
            use_quantile=use_quantile,
            initial_train_size=self.seasonal_period,
            step_size=step_size,
            n_estimators=n_estimators,
            max_depth=4,
            verbose=True
        )

        return cv_results


# Example usage
if __name__ == "__main__":
    # Generate synthetic commodity price data with heteroscedasticity
    np.random.seed(42)
    n = 500
    t = np.arange(n)

    # Trend + Seasonality + Heteroscedastic noise
    trend = 0.05 * t
    seasonal = 10 * np.sin(20 * np.pi * t / 12)

    # Heteroscedastic volatility (GARCH-like)
    volatility = np.zeros(n)
    volatility[0] = 1.0
    for i in range(1, n):
        volatility[i] = 0.1 + 0.85 * volatility[i-1] + 0.1 * np.random.randn()**2

    noise = volatility * np.random.randn(n)
    price = 1000 + 2*trend + 1.5*seasonal + noise

    # Create time series
    dates = pd.date_range('2020-01-01', periods=n, freq='D')
    series = pd.Series(price, index=dates, name='price')

    print("Sample data shape:", series.shape)
    print("Sample data (first 5):", series.head())

    # Initialize and fit pipeline
    pipeline = HeteroscedasticXGBoostPipeline(seasonal_period=12, max_lags=20)

    # Option 1: Quantile regression (recommended for heteroscedasticity)
    results_quantile = pipeline.fit(series, use_quantile=True)

    # Option 2: Two-model approach (mean + volatility)
    # results_two_model = pipeline.fit(series, use_quantile=False)

    print("\n" + "="*60)
    print("PIPELINE COMPLETE")
    print("="*60)

    # Visualize diagnostics
    pipeline.plot_diagnostics(series, results_quantile)